Подведите результаты эксперимента в экселе по следующим данным: ab_stats.csv - Google Диск: Стат значимо ли отличается ARPPU в двух группах ? Какие рекомендации дадите менеджеру?

In [1]:
import numpy as np
import pandas as pd

import scipy

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [7]:
data_path = "./ab_stats.csv"
data = pd.read_csv(data_path)
print("Размеры датасета", data.shape)
data.head(6)

Размеры датасета (23652, 5)


,revenue,num_purchases,purchase,ab_group,av_site visit
0,0.0,0,0,A,9.040174
1,0.0,0,0,A,4.811628
2,0.0,0,0,A,7.342623
3,0.0,0,0,A,7.744581
4,0.0,0,0,A,10.511814
5,0.0,0,0,A,9.578727


1. `revenue`: доход.
2. `num_purchases`: количество покупок.
3. `purchase`: индикатор покупки (0 или 1).
4. `ab_group`: группа (A или B).
5. `av_site visit`: время посещения сайта.

# 2

Мы хотим провести А/Б-тест для трех источников трафика. Нынешняя конверсия равна 5%, мы ожидаем прирост в 0,2%. Уровень доверия 97% и уровень мощности 87%. Всего на наш продукт заходит 40 000 пользователей в месяц.

# 3
За сколько дней мы сможем протестировать гипотезу? И что вы можете посоветовать по результатам подсчета?
# 3.1 

Допустим в задаче 3 нет проблемы с количеством посетителей на сайт, тогда подведите результаты тестирования, если у нас следующие результаты по количеству конверсии:


    1. 25 000 
    2. 30 000 
    3. 32 000

Проведем статистический тест (например, Z-тест) для проверки гипотезы о значимости различий.

In [12]:
from statsmodels.stats.power import zt_ind_solve_power
import numpy as np

# Исходные данные для расчета
current_conversion = 0.05  # Текущая конверсия
expected_improvement = 0.002  # Ожидаемое улучшение
alpha = 0.03  # Уровень значимости (1 - уровень доверия)
power = 0.87  # Уровень мощности

# Конверсия в варианте B
alternative_conversion = current_conversion + expected_improvement

# Расчет эффекта (effect size)
effect_size = (alternative_conversion - current_conversion) / (
    (current_conversion * (1 - current_conversion)) ** 0.5
)

# Расчет минимального размера выборки для одной группы
sample_size_per_group = zt_ind_solve_power(
    effect_size=effect_size, alpha=alpha, power=power, alternative="two-sided"
)
sample_size_per_group = int(np.ceil(sample_size_per_group))  # Округление до целого числа

# Всего пользователей в месяц
total_users_per_month = 40000

# Пользователи, которые могут участвовать в тесте (делим на 2 группы)
daily_users_per_group = total_users_per_month / 30 / 2

# Количество дней, необходимых для теста
required_days = np.ceil(sample_size_per_group / daily_users_per_group)

print(f"Sample size per group: {sample_size_per_group} \n Required days: {required_days}")

Sample size per group: 258087 
 Required days: 388.0


### Рекомендации:
**Долгое время тестирования:**
Такой длительный период (более года) может быть неэффективным для принятия решений.
Рассмотрите возможность увеличения трафика на тестируемые варианты (например, привлечение большего числа пользователей из конкретных источников).

**Пересмотр ожидаемого эффекта:**
Прирост в 0.2% (от 5% до 5.2%) — это очень небольшой эффект. Если вы считаете возможным допустить большее отклонение, например, 0.5%, то это сократит необходимый объем выборки.

**Сегментация:**
Вы можете протестировать гипотезу на сегментах аудитории (например, только один источник трафика) для ускорения анализа.

In [26]:
# Результаты тестирования: количество конверсий в группах
conversions_A = 25000  # Группа A
conversions_B1 = 30000  # Группа B1
conversions_B2 = 32000  # Группа B2

# Размер выборок для групп (в каждой группе одинаковое количество участников)
sample_size_A = sample_size_per_group
sample_size_B1 = sample_size_per_group
sample_size_B2 = sample_size_per_group

# Расчет конверсий для каждой группы
conversion_rate_A = conversions_A / sample_size_A
conversion_rate_B1 = conversions_B1 / sample_size_B1
conversion_rate_B2 = conversions_B2 / sample_size_B2

# Z-тест для сравнения группы A с группой B1 и B2
from statsmodels.stats.proportion import proportions_ztest

# Группа A vs Группа B1
stat_B1, pval_B1 = proportions_ztest([conversions_A, conversions_B1], [sample_size_A, sample_size_B1])

# Группа A vs Группа B2
stat_B2, pval_B2 = proportions_ztest([conversions_A, conversions_B2], [sample_size_A, sample_size_B2])

# Итоговые конверсии и результаты теста
print(f"Conversion Rate A{conversion_rate_A} \n",
        f"Conversion Rate B1: {conversion_rate_B1} \n",
        f"Conversion Rate B2: {conversion_rate_B2} \n",
        f"Z-Statistic A vs B1: {stat_B1} \n",
        f"P-Value A vs B1: {pval_B1} \n",
        f"Z-Statistic A vs B2: {stat_B2} \n",
        f"P-Value A vs B2: {pval_B2}")


Conversion Rate A0.09686656050091635 
 Conversion Rate B1: 0.11623987260109962 
 Conversion Rate B2: 0.12398919744117294 
 Z-Statistic A vs B1: -22.55559639262107 
 P-Value A vs B1: 1.1833335767987e-112 
 Z-Statistic A vs B2: -31.08637132270179 
 P-Value A vs B2: 3.6810782381434956e-212


**Конверсии:**

Группа A: 9.69% (25,000 / 258,087).

Группа B1: 11.62% (30,000 / 258,087).

Группа B2: 12.40% (32,000 / 258,087).

**Сравнение A vs B1:**

Z-статистика: -22.56.

P-значение: значительно ниже уровня значимости (0.03)

Вывод: Различия между группами A и B1 статистически значимы.

**Сравнение A vs B2:**

Z-статистика: -31.09.

P-значение: значительно ниже уровня значимости (0.03)

Вывод: Различия между группами A и B2 статистически значимы.

**Рекомендации:**

Группы B1 и B2 показали значительно лучшую конверсию по сравнению с группой A.

Между B1 и B2 также можно провести тест, чтобы определить, есть ли значимые различия, если важно выбрать лучшее из двух.

Учитывая статистическую значимость, есть основание для перехода на новую версию, представленную в группах B1 или B2.

# 4.
Вы решили сравнивать метрику CPA в двух группах. Размер выборки - 2350 элементов в каждой группе. 
    
Для проверки нормальности распределения на выборке в 2350 наблюдений применили , критерий Шапиро-Уилка и получили p-value, равный 0.00002, alpha = 5%. 
   
Какой бы вывод мы могли сделать в данном случае? 
    

В этом случае какой статистический критерий для проверки первоначальной гипотезы тут лучше всего подойдёт и почему ?

In [37]:
from scipy.stats import shapiro, mannwhitneyu
import numpy as np

# Генерация данных для примера
np.random.seed(42)  # Для воспроизводимости
group_A = np.random.exponential(scale=2.0, size=2350)  # Данные группы A (нормальные)
group_B = np.random.exponential(scale=2.1, size=2350)  # Данные группы B (нормальные)

# Проверка нормальности распределения с помощью теста Шапиро-Уилка
stat_A, pval_A = shapiro(group_A)
stat_B, pval_B = shapiro(group_B)

# Применение U-теста Манна-Уитни для сравнения двух групп
u_stat, p_value_u = mannwhitneyu(group_A, group_B, alternative="two-sided")

print(f"Shapiro-Wilk Test Group A: \n Statistic: {stat_A}, \n P-Value: {pval_A} \n", 
      f"\n Shapiro-Wilk Test Group B: \n Statistic: {stat_B}, \n P-Value: {pval_B} \n",
      f"\n Mann-Whitney U Test: \n U-Statistic: {u_stat}, \n P-Value: {p_value_u}")

Shapiro-Wilk Test Group A: 
 Statistic: 0.8211322394153019, 
 P-Value: 2.4476946053526133e-45 
 
 Shapiro-Wilk Test Group B: 
 Statistic: 0.8199248918501773, 
 P-Value: 1.9182246955873275e-45 
 
 Mann-Whitney U Test: 
 U-Statistic: 2705915.0, 
 P-Value: 0.2341792283274131


**Вывод:** нет статистически значимой разницы между группами A и B по сравнению метрики CPA.

Поскольку данные не распределены нормально, использование параметрических тестов, таких как t-тест, может быть некорректным. Для проверки гипотезы лучше использовать непараметрические тесты, например, **U-тест Манна-Уитни**

# 5. 

Мы провели АБ-тест на увеличение average timespent  per user. По итогам тестирования мы получили следующие данные. Является ли результат статистически значимым с уровнем доверия 80%? Какую версию мы выкатим на продакшн? 

A) Средняя - 360, отклонение - 40, количество - 9802 

B) Средняя - 352, отклонение - 58, количество - 9789

In [39]:
# Импортируем недостающую функцию для t-теста
from scipy.stats import ttest_ind_from_stats

mean_A, std_A, n_A = 360, 40, 9802
mean_B, std_B, n_B = 352, 58, 9789

# Проведение t-теста на основе заданных данных
t_stat, p_value = ttest_ind_from_stats(
    mean1=mean_A, std1=std_A, nobs1=n_A,
    mean2=mean_B, std2=std_B, nobs2=n_B,
    equal_var=False  # Не предполагаем равенство дисперсий
)

# Вывод результатов
print(f"T-Statistic: {t_stat} \n",
        f"P-Value: {p_value} \n",
    f"Significance_Level (Alpha): 0.2 \n",
    f"Statistically Significant: {p_value < 0.2}")

T-Statistic: 11.236630785707332 
 P-Value: 3.397782878412232e-29 
 Significance_Level (Alpha): 0.2 
 Statistically Significant: True


Результат является статистически значимым при уровне доверия 80%.
Есть достаточные основания утверждать, что среднее время, проведённое пользователями, отличается между группами A и B.

**Рекомендация:**
Версия на продакшн: Группа A, так как её среднее время (360 секунд) больше, чем у группы B (352 секунды). Выбор этой версии приведёт к увеличению времени, проводимого пользователями на платформе. ​

# 6.

Создайте техническую архитектуру проекта по аб тестированию продукта он-лайн кинотеатра с учетом кросс-девайс аналитики по следующей гипотезе:
    
Если договориться с банком о 99% кэшбэке на подписку первого месяца, то это повысит конверсию в подписку на 30%, благодаря упрощенному принятию решения со стороны пользователя. 
    
На схеме необходимо отобразить:
    
    1. Управленческий процесс по договоренностям с внешними партнерами 
    2. Архитектуру данных с указанием систем, из которых будем скачивать данные 
    3. Внутрикомандное взаимодействие

### Техническая архитектура проекта A/B-тестирования с учетом кросс-девайс аналитики

**1. Управленческий процесс:**

Договоренности с банком о 99% кэшбэке.
Настройка механизма взаимодействия через API.

**2.Архитектура данных:**

Источники данных:

* Платформа кинотеатра (CMS, CRM).
* Финансовый API банка.
* Системы кросс-девайс аналитики (Google Analytics, Amplitude).

Хранилище данных:

* Data Lake для сырой информации.
* Data Warehouse для аналитики.

Инструменты:

* SQL/BI-платформы, Python для анализа.

**3. Внутрикомандное взаимодействие:**

* Продакт-менеджер: постановка целей.
* Аналитики: сбор и анализ данных.
* Разработчики: интеграция API.
* Маркетинг: настройка трекинга и продвижение гипотезы.